In [0]:
%sql
CREATE TABLE customer_profiles (
    customer_id BIGINT,
    email STRING,
    phone STRING,
    profile STRING   -- JSON stored as STRING
)
USING DELTA;


In [0]:
%sql
INSERT INTO customer_profiles VALUES
(
  1001,
  'john.doe@example.com',
  '+1-555-1111',
  '{
     "first_name": "John",
     "last_name": "Doe",
     "gender": "Male",
     "address": {
       "street": "12 Main Street",
       "city": "New York",
       "country": "USA"
     }
   }'
),
(
  1002,
  'jane.smith@example.com',
  '+44-777-2222',
  '{
     "first_name": "Jane",
     "last_name": "Smith",
     "gender": "Female",
     "address": {
       "street": "45 Baker Street",
       "city": "London",
       "country": "UK"
     }
   }'
),
(
  1003,
  'arjun.patel@example.com',
  '+91-999-3333',
  '{
     "first_name": "Arjun",
     "last_name": "Patel",
     "gender": "Male",
     "address": {
       "street": "101 MG Road",
       "city": "Bengaluru",
       "country": "India"
     }
   }'
),
(
  1004,
  'maria.garcia@example.com',
  '+34-888-4444',
  '{
     "first_name": "Maria",
     "last_name": "Garcia",
     "gender": "Female",
     "address": {
       "street": "22 Gran Via",
       "city": "Madrid",
       "country": "Spain"
     }
   }'
),
(
  1005,
  'li.wei@example.com',
  '+86-666-5555',
  '{
     "first_name": "Li",
     "last_name": "Wei",
     "gender": "Male",
     "address": {
       "street": "88 Nanjing Road",
       "city": "Shanghai",
       "country": "China"
     }
   }'
);


num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
select * from customer_profiles

customer_id,email,phone,profile
1001,john.doe@example.com,+1-555-1111,"{ ""first_name"": ""John"", ""last_name"": ""Doe"", ""gender"": ""Male"", ""address"": { ""street"": ""12 Main Street"", ""city"": ""New York"", ""country"": ""USA"" } }"
1002,jane.smith@example.com,+44-777-2222,"{ ""first_name"": ""Jane"", ""last_name"": ""Smith"", ""gender"": ""Female"", ""address"": { ""street"": ""45 Baker Street"", ""city"": ""London"", ""country"": ""UK"" } }"
1003,arjun.patel@example.com,+91-999-3333,"{ ""first_name"": ""Arjun"", ""last_name"": ""Patel"", ""gender"": ""Male"", ""address"": { ""street"": ""101 MG Road"", ""city"": ""Bengaluru"", ""country"": ""India"" } }"
1004,maria.garcia@example.com,+34-888-4444,"{ ""first_name"": ""Maria"", ""last_name"": ""Garcia"", ""gender"": ""Female"", ""address"": { ""street"": ""22 Gran Via"", ""city"": ""Madrid"", ""country"": ""Spain"" } }"
1005,li.wei@example.com,+86-666-5555,"{ ""first_name"": ""Li"", ""last_name"": ""Wei"", ""gender"": ""Male"", ""address"": { ""street"": ""88 Nanjing Road"", ""city"": ""Shanghai"", ""country"": ""China"" } }"


In [0]:
%sql
describe customer_profiles

col_name,data_type,comment
customer_id,bigint,null
email,string,null
phone,string,null
profile,string,null


In [0]:
%sql
select customer_id,profile:first_name,profile:address:city from customer_profiles

customer_id,first_name,city
1001,John,New York
1002,Jane,London
1003,Arjun,Bengaluru
1004,Maria,Madrid
1005,Li,Shanghai


In [0]:
%sql
select from_json(profile) AS profile_struct from customer_profiles

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4685471064123664>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'select from_json(profile) AS profile_struct from customer_profiles\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:194, in SqlMagic.sql(self, line, cell)
    188 except BaseException as e:
    189    

In [0]:
%sql
select profile from customer_profiles limit 1

profile
"{ ""first_name"": ""John"", ""last_name"": ""Doe"", ""gender"": ""Male"", ""address"": { ""street"": ""12 Main Street"", ""city"": ""New York"", ""country"": ""USA"" } }"


In [0]:
%sql
create or replace temp view parse_customers AS select customer_id,from_json(profile,schema_of_json('{ "first_name": "John", "last_name": "Doe", "gender": "Male", "address": { "street": "12 Main Street", "city": "New York", "country": "USA" } }')) AS profile_struct from customer_profiles;

select * from parse_customers

customer_id,profile_struct
1001,"List(List(New York, USA, 12 Main Street), John, Male, Doe)"
1002,"List(List(London, UK, 45 Baker Street), Jane, Female, Smith)"
1003,"List(List(Bengaluru, India, 101 MG Road), Arjun, Male, Patel)"
1004,"List(List(Madrid, Spain, 22 Gran Via), Maria, Female, Garcia)"
1005,"List(List(Shanghai, China, 88 Nanjing Road), Li, Male, Wei)"


In [0]:
%sql
describe parse_customers

col_name,data_type,comment
customer_id,bigint,null
profile_struct,"struct,first_name:string,gender:string,last_name:string>",null


In [0]:
select customer_id, profile_struct.address.city, profile_struct.first_name from parse_customers

customer_id,city,first_name
1001,New York,John
1002,London,Jane
1003,Bengaluru,Arjun
1004,Madrid,Maria
1005,Shanghai,Li


In [0]:
%sql
create or replace temp view customer_data AS select customer_id,profile_struct.* from parse_customers;

select * from customer_data

customer_id,address,first_name,gender,last_name
1001,"List(New York, USA, 12 Main Street)",John,Male,Doe
1002,"List(London, UK, 45 Baker Street)",Jane,Female,Smith
1003,"List(Bengaluru, India, 101 MG Road)",Arjun,Male,Patel
1004,"List(Madrid, Spain, 22 Gran Via)",Maria,Female,Garcia
1005,"List(Shanghai, China, 88 Nanjing Road)",Li,Male,Wei


In [0]:
drop table customer_profiles_array

In [0]:
%sql
CREATE TABLE customer_profiles_array (
    customer_id INT,
    profiles ARRAY<
        STRUCT<
            first_name: STRING,
            last_name: STRING,
            gender: STRING,
            address: STRUCT<
                street: STRING,
                city: STRING,
                country: STRING
            >
        >
    >
)
USING DELTA;


In [0]:
%sql
INSERT INTO customer_profiles_array VALUES
(
  1,
  array(
    struct(
      'John', 'Doe', 'Male',
      struct('12 MG Road', 'Bangalore', 'India')
    ),
    struct(
      'Johnny', 'Doe', 'Male',
      struct('45 Park Ave', 'New York', 'USA')
    )
  )
),
(
  2,
  array(
    struct(
      'Priya', 'Sharma', 'Female',
      struct('8 Link Street', 'Delhi', 'India')
    )
  )
),
(
  3,
  array(
    struct(
      'Alex', 'Brown', 'Male',
      struct('77 King St', 'London', 'UK')
    ),
    struct(
      'Alexa', 'Brown', 'Female',
      struct('77 King St', 'London', 'UK')
    )
  )
),
(
  4,
  array(
    struct(
      'Maria', 'Garcia', 'Female',
      struct('Calle 10', 'Madrid', 'Spain')
    )
  )
),
(
  5,
  array(
    struct(
      'Chen', 'Wei', 'Male',
      struct('88 Lotus Rd', 'Shanghai', 'China')
    )
  )
);


num_affected_rows,num_inserted_rows
5,5


In [0]:
select * from customer_profiles_array 

customer_id,profiles
1,"List(List(John, Doe, Male, List(12 MG Road, Bangalore, India)), List(Johnny, Doe, Male, List(45 Park Ave, New York, USA)))"
2,"List(List(Priya, Sharma, Female, List(8 Link Street, Delhi, India)))"
3,"List(List(Alex, Brown, Male, List(77 King St, London, UK)), List(Alexa, Brown, Female, List(77 King St, London, UK)))"
4,"List(List(Maria, Garcia, Female, List(Calle 10, Madrid, Spain)))"
5,"List(List(Chen, Wei, Male, List(88 Lotus Rd, Shanghai, China)))"


In [0]:
select customer_id,explode(profiles) from customer_profiles_array;

customer_id,col
1,"List(John, Doe, Male, List(12 MG Road, Bangalore, India))"
1,"List(Johnny, Doe, Male, List(45 Park Ave, New York, USA))"
2,"List(Priya, Sharma, Female, List(8 Link Street, Delhi, India))"
3,"List(Alex, Brown, Male, List(77 King St, London, UK))"
3,"List(Alexa, Brown, Female, List(77 King St, London, UK))"
4,"List(Maria, Garcia, Female, List(Calle 10, Madrid, Spain))"
5,"List(Chen, Wei, Male, List(88 Lotus Rd, Shanghai, China))"


In [0]:
select customer_id, collect_set(profiles.gender) AS gender_set
from customer_profiles_array group by customer_id

customer_id,gender_set
1,"List(List(Male, Male))"
2,List(List(Female))
3,"List(List(Male, Female))"
4,List(List(Female))
5,List(List(Male))


In [0]:
select customer_id, collect_set(profiles.gender) AS before_flatten,array_distinct(flatten(collect_set(profiles.gender)))
from customer_profiles_array group by customer_id

customer_id,before_flatten,array_distinct(flatten(collect_set(profiles.gender)))
1,"List(List(Male, Male))",List(Male)
2,List(List(Female)),List(Female)
3,"List(List(Male, Female))","List(Male, Female)"
4,List(List(Female)),List(Female)
5,List(List(Male)),List(Male)


# Higher Order Functions and SQL UDF

In [0]:
select customer_id,profiles,filter(profiles,i -> i.gender = "Male") as mutliple_copy from customer_profiles_array

customer_id,profiles,mutliple_copy
1,"List(List(John, Doe, Male, List(12 MG Road, Bangalore, India)), List(Johnny, Doe, Male, List(45 Park Ave, New York, USA)))","List(List(John, Doe, Male, List(12 MG Road, Bangalore, India)), List(Johnny, Doe, Male, List(45 Park Ave, New York, USA)))"
2,"List(List(Priya, Sharma, Female, List(8 Link Street, Delhi, India)))",List()
3,"List(List(Alex, Brown, Male, List(77 King St, London, UK)), List(Alexa, Brown, Female, List(77 King St, London, UK)))","List(List(Alex, Brown, Male, List(77 King St, London, UK)))"
4,"List(List(Maria, Garcia, Female, List(Calle 10, Madrid, Spain)))",List()
5,"List(List(Chen, Wei, Male, List(88 Lotus Rd, Shanghai, China)))","List(List(Chen, Wei, Male, List(88 Lotus Rd, Shanghai, China)))"


In [0]:
select * from (select customer_id,profiles,filter(profiles,i -> i.gender = "Male") as mutliple_copy from customer_profiles_array) where size (mutliple_copy) > 0

customer_id,profiles,mutliple_copy
1,"List(List(John, Doe, Male, List(12 MG Road, Bangalore, India)), List(Johnny, Doe, Male, List(45 Park Ave, New York, USA)))","List(List(John, Doe, Male, List(12 MG Road, Bangalore, India)), List(Johnny, Doe, Male, List(45 Park Ave, New York, USA)))"
3,"List(List(Alex, Brown, Male, List(77 King St, London, UK)), List(Alexa, Brown, Female, List(77 King St, London, UK)))","List(List(Alex, Brown, Male, List(77 King St, London, UK)))"
5,"List(List(Chen, Wei, Male, List(88 Lotus Rd, Shanghai, China)))","List(List(Chen, Wei, Male, List(88 Lotus Rd, Shanghai, China)))"


# UDF

In [0]:
create or replace function get_url(first_name string,last_name string)
returns string
return concat(first_name,"_",last_name)

In [0]:
create or replace temp view exploded_profile AS select customer_id,explode(profiles) AS exploded_profiles from customer_profiles_array;

In [0]:
select customer_id,get_url(exploded_profiles.first_name,exploded_profiles.last_name) from exploded_profile

customer_id,"get_url(exploded_profiles.first_name,exploded_profiles.last_name)"
1,John_Doe
1,Johnny_Doe
2,Priya_Sharma
3,Alex_Brown
3,Alexa_Brown
4,Maria_Garcia
5,Chen_Wei
